In [1]:
####################################################################
# Scenario0: static하게 앙상블
# 모든 컴비네이션으로 경우의 수를 완탐해보고 마지막 출구보다 성능 좋은것 유무
####################################################################
# Scenario1: Entropy vs Temperature Scaling + Entropy 
# make dynamic ensemble of models which entropy is less than threshold
####################################################################
# Scenario2: MC Dropout -> find confident EE -> static ensemble 
#step1: make MC Dropout model
#step2: find confident EE from experiment
#step3: sum of softmax vector of each good model(under threshold) -> final inference from softmax vector sum
####################################################################
# Scenario3: train new block to choose which exit to inference 
# (JUST SCENARIO, NOT VERIFIED)
####################################################################

##### 모델 로드 및 캐시 저장

In [2]:
import torch
import torch.nn.functional as F
from torchvision import datasets
from tqdm import tqdm
from itertools import combinations
import numpy as np
from scipy.stats import entropy
from collections import defaultdict
from temperature_scaling import TemperatureScaling
from matrix_scaling import MatrixScaling
from Dloaders import Dloaders

IMG_SIZE = 224
device = 'cuda' if torch.cuda.is_available() else 'cpu'
dataset_name = {'cifar10':datasets.CIFAR10, 'cifar100':datasets.CIFAR100,'imagenet':None}
dataset_outdim = {'cifar10':10, 'cifar100':100,'imagenet':1000}
################ 0. Hyperparameters ##########################
batch_size = 1024
data_choice='cifar100'
model_choice = 'resnet' # ['vit', 'resnet']
exit_num=11
cache_file_path = f'cache_result_{model_choice}_{data_choice}.pt'
precision_each_label = f'cache_precision_each_label_{data_choice}.pt'
choosen_exit_distribution = f'cache_choosen_exit_distribution_{data_choice}.pt'
exp_result_path = f'cache_exp_{model_choice}_{data_choice}.pt'
t_scalers_path = f"models/{model_choice}/{data_choice}/temperature_scaler.pth"
m_scalers_path = f"models/{model_choice}/{data_choice}/matrix_scaler.pth"

exp_save_file = dict()

2025-01-03 21:18:39.513500: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-03 21:18:40.800262: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
dloaders=Dloaders(data_choice=data_choice,batch_size=batch_size,IMG_SIZE=IMG_SIZE)
train_loader,test_loader = dloaders.get_loaders()

# 저장한 파일을 다시 불러오기
output_tensor = torch.load(cache_file_path).to(device)
_, test_dataset = dloaders.get_datasets()
labels_list = test_dataset.targets
labels=torch.tensor(labels_list).to(device)
# 데이터 확인
print(output_tensor.shape)  # <class 'list'>
print(output_tensor[0].shape)  # torch.Size([10000, 100])

Files already downloaded and verified
Files already downloaded and verified


/tmp/ipykernel_31073/3583907403.py:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  output_tensor = torch.load(cache_file_path).to(device)


torch.Size([11, 10000, 100])
torch.Size([10000, 100])


##### 이전 출구들 전부를 앙상블

In [4]:
prefix_sum_ensemble_acc = []
for i in range(exit_num):
    new_output_list = output_tensor[:i + 1,:,:]
    ensemble_logits = new_output_list.sum(axis=0)
    ensemble_probabilities = F.softmax(ensemble_logits, dim=1)
    _, predicted_labels = torch.max(ensemble_probabilities, dim=1)
    correct_predictions = (predicted_labels == labels).sum().item()
    total_predictions = labels.size(0)
    accuracy = correct_predictions / total_predictions * 100
    prefix_sum_ensemble_acc.append(round(accuracy,2))
exp_save_file['prefix_sum_ensemble_acc'] = prefix_sum_ensemble_acc
print(*prefix_sum_ensemble_acc, sep = ', ')

37.8, 43.54, 55.63, 63.15, 68.88, 72.54, 75.53, 78.83, 81.1, 83.87, 85.18


##### 모든 경우의 수에 대해서 소프트맥스 전에 정적 앙상블 하고 소프트맥스 진행

In [5]:
combinations_list = []
for i in range(2, exit_num+1):combinations_list.extend(combinations(range(exit_num), i))

In [6]:
# sum before softmax (best acc: 87.75)
bef_softmax = dict()
for choosed_exits in combinations_list:
    new_output_list = output_tensor[choosed_exits,:,:]
    ensemble_logits = new_output_list.sum(axis=0)
    ensemble_probabilities = F.softmax(ensemble_logits, dim=1)
    _, predicted_labels = torch.max(ensemble_probabilities, dim=1)
    correct_predictions = (predicted_labels == labels).sum().item()
    total_predictions = labels.size(0)
    accuracy = correct_predictions / total_predictions * 100
    bef_softmax[choosed_exits]=accuracy

tmp = sorted(list(bef_softmax.items()), key=lambda x: x[1], reverse=True)
Static = tmp[0][1]
print(tmp)

[((4, 10), 86.52), ((3, 10), 86.45), ((6, 10), 86.45), ((3, 7, 10), 86.22), ((4, 7, 10), 86.2), ((5, 10), 86.18), ((7, 10), 86.18), ((4, 8, 10), 86.18), ((2, 6, 10), 86.16), ((6, 7, 10), 86.15), ((3, 8, 10), 86.11999999999999), ((4, 6, 9, 10), 86.11999999999999), ((4, 9, 10), 86.11), ((6, 8, 10), 86.09), ((3, 6, 9, 10), 86.09), ((4, 7, 9, 10), 86.09), ((4, 6, 8, 9, 10), 86.09), ((3, 6, 10), 86.08), ((3, 5, 10), 86.06), ((2, 4, 7, 9, 10), 86.06), ((3, 6, 8, 9, 10), 86.05000000000001), ((3, 4, 6, 8, 9, 10), 86.05000000000001), ((8, 10), 86.04), ((6, 8, 9, 10), 86.04), ((2, 4, 9, 10), 86.03), ((2, 6, 7, 9, 10), 86.03), ((3, 4, 7, 9, 10), 86.03), ((4, 6, 7, 9, 10), 86.03), ((3, 4, 7, 10), 86.02), ((3, 6, 8, 10), 86.02), ((5, 8, 9, 10), 86.02), ((1, 3, 6, 8, 9, 10), 86.02), ((2, 5, 6, 8, 9, 10), 86.02), ((2, 7, 10), 86.00999999999999), ((3, 7, 9, 10), 86.00999999999999), ((5, 6, 8, 9, 10), 86.00999999999999), ((3, 4, 6, 7, 9, 10), 86.00999999999999), ((3, 4, 7, 8, 9, 10), 86.00999999999999)

##### 모든 경우의 수에 대해서 소프트맥스 하고 정적 앙상블 하고 맥스값으로 추론 진행

In [7]:
# sum after softmax (best acc: 87.75)
aft_softmax = dict()
for choosed_exits in combinations_list:
    new_output_list = output_tensor[choosed_exits,:,:]
    softmax_vector_list = F.softmax(new_output_list, dim=2)
    ensemble_probabilities = softmax_vector_list.sum(axis=0)
    _, predicted_labels = torch.max(ensemble_probabilities, dim=1)
    correct_predictions = (predicted_labels == labels).sum().item()
    total_predictions = labels.size(0)
    accuracy = correct_predictions / total_predictions * 100
    aft_softmax[choosed_exits]=accuracy

tmp = sorted(list(aft_softmax.items()), key=lambda x: x[1], reverse=True)
Static = max(Static, tmp[0][1])
print(tmp)

[((4, 10), 86.44), ((6, 10), 86.42999999999999), ((0, 6, 10), 86.29), ((2, 10), 86.25), ((3, 10), 86.25), ((5, 10), 86.22999999999999), ((1, 6, 10), 86.22999999999999), ((8, 10), 86.19), ((0, 8, 10), 86.18), ((3, 6, 9, 10), 86.15), ((0, 6, 8, 9, 10), 86.15), ((0, 10), 86.14), ((0, 4, 10), 86.14), ((1, 10), 86.13), ((1, 8, 10), 86.13), ((4, 8, 10), 86.13), ((6, 8, 9, 10), 86.11), ((0, 3, 6, 9, 10), 86.1), ((7, 10), 86.09), ((3, 7, 9, 10), 86.09), ((1, 6, 8, 9, 10), 86.09), ((6, 8, 10), 86.08), ((1, 3, 6, 9, 10), 86.08), ((0, 1, 5, 6, 8, 9, 10), 86.08), ((2, 7, 9, 10), 86.07000000000001), ((5, 6, 8, 9, 10), 86.06), ((0, 3, 7, 9, 10), 86.05000000000001), ((0, 1, 6, 8, 9, 10), 86.05000000000001), ((1, 3, 7, 9, 10), 86.04), ((2, 6, 8, 9, 10), 86.04), ((3, 6, 8, 9, 10), 86.04), ((0, 2, 6, 8, 9, 10), 86.04), ((1, 4, 8, 10), 86.03), ((0, 5, 10), 86.02), ((3, 7, 10), 86.02), ((3, 8, 10), 86.02), ((0, 2, 7, 9, 10), 86.02), ((1, 3, 6, 8, 9, 10), 86.02), ((0, 7, 10), 86.00999999999999), ((3, 6, 8,

In [8]:
exp_save_file['static_ensemble_acc'] = Static

##### 동적 앙상블: 모든 exit에서 특정 엔트로피 이하인 것들을 소프트맥스 전에 앙상블 하고 소프트맥스 진행 한 샘플에서 임계값보다 작은게 없으면 제일 뒤의 블록 선택만으로 추론

In [9]:
# 모든 exit에서 특정 엔트로피 이하인 것들을 모아서 앙상블을 해본다.
# case 2가지 소프트맥스 전에 합칠지 후에 합칠지;;

step_range = 10000
aft_sftmx = F.softmax(output_tensor,dim=2)
entropy_array= torch.tensor(entropy(aft_sftmx.to('cpu'), base=exit_num, axis=2))

min_entropy = entropy_array.min()
median_entropy = torch.median(torch.tensor(entropy_array))

step_size = (median_entropy - min_entropy) / step_range
d_bef_softmax=(0,0);d_aft_softmax=(0,0)
for mul in tqdm(range(step_range)):
    threshold = min_entropy + mul * step_size
    mask = (entropy_array[:, :] <= threshold).to(device)
    
    column_sums = mask.sum(dim=0)  
    zero_columns = (column_sums == 0)

    last_row = torch.zeros_like(mask)
    last_row[-1, :] = 1
    mask = mask | (last_row & zero_columns)

    mask.unsqueeze_(dim=2)
    masked_array=mask*output_tensor

    # sum before softmax (best acc: 87.75)
    ensemble_logits = masked_array.sum(axis=0)
    ensemble_probabilities = F.softmax(ensemble_logits, dim=1)
    _, predicted_labels = torch.max(ensemble_probabilities, dim=1)
    correct_predictions = (predicted_labels == labels).sum().item()
    total_predictions = labels.size(0)
    accuracy = correct_predictions / total_predictions * 100
    d_bef_softmax=max(d_bef_softmax,(threshold,accuracy),key=lambda x:x[1])

    # sum after softmax (best acc: 87.75)
    softmax_vector_list = F.softmax(masked_array, dim=2)
    ensemble_probabilities = softmax_vector_list.sum(axis=0)
    _, predicted_labels = torch.max(ensemble_probabilities, dim=1)
    correct_predictions = (predicted_labels == labels).sum().item()
    total_predictions = labels.size(0)
    accuracy = correct_predictions / total_predictions * 100
    d_aft_softmax=max(d_aft_softmax,(threshold,accuracy),key=lambda x:x[1])
print(f"d_bef_softmax: {d_bef_softmax}\nd_aft_softmax: {d_aft_softmax}")
Dynamic_No_Scaling = max(d_bef_softmax[1], d_aft_softmax[1])

/tmp/ipykernel_31073/3795230587.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  median_entropy = torch.median(torch.tensor(entropy_array))
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:06<00:00, 1441.29it/s]

d_bef_softmax: (tensor(0.0011), 86.29)
d_aft_softmax: (tensor(0.0011), 86.29)


##### 동적 앙상블: 모든 exit에서 특정 엔트로피 이하인 것들을 소프트맥스 전에 앙상블 하고 소프트맥스 진행 한 샘플에서 임계값보다 작은게 없으면 제일 엔트로피 작은 것을 선택해서 그것의 선택만으로 추론

In [10]:
# 모든 exit에서 특정 엔트로피 이하인 것들을 모아서 앙상블을 해본다.
# case 2가지 소프트맥스 전에 합칠지 후에 합칠지;;

step_range = 10000
aft_sftmx = F.softmax(output_tensor,dim=2)
entropy_array= torch.tensor(entropy(aft_sftmx.to('cpu'), base=exit_num, axis=2)).to(device)

min_entropy = entropy_array.min()
median_entropy = torch.median(torch.tensor(entropy_array))

step_size = (median_entropy - min_entropy) / step_range
d_bef_softmax=(0,0);d_aft_softmax=(0,0)

########################################
# 각 열의 최소값 인덱스 찾기
min_indices = entropy_array.argmin(dim=0)
# 2D 행렬 생성 및 초기화 (모든 값 0)
result_matrix = torch.zeros_like(entropy_array.int())
# 최소값 인덱스 위치에 1 설정
for col, row in enumerate(min_indices):
    result_matrix[row, col] = 1
########################################

for mul in tqdm(range(step_range)):
    threshold = min_entropy + mul * step_size
    mask = (entropy_array[:, :] <= threshold).to(device)
    
    column_sums = mask.sum(dim=0)
    zero_columns = (column_sums == 0)
    mask = mask | (result_matrix & zero_columns)

    mask.unsqueeze_(dim=2)
    masked_array=mask*output_tensor

    # sum before softmax (best acc: 87.75)
    ensemble_logits = masked_array.sum(axis=0)
    ensemble_probabilities = F.softmax(ensemble_logits, dim=1)
    _, predicted_labels = torch.max(ensemble_probabilities, dim=1)
    correct_predictions = (predicted_labels == labels).sum().item()
    total_predictions = labels.size(0)
    accuracy = correct_predictions / total_predictions * 100
    d_bef_softmax=max(d_bef_softmax,(threshold,accuracy),key=lambda x:x[1])

    # sum after softmax (best acc: 87.75)
    softmax_vector_list = F.softmax(masked_array, dim=2)
    ensemble_probabilities = softmax_vector_list.sum(axis=0)
    _, predicted_labels = torch.max(ensemble_probabilities, dim=1)
    correct_predictions = (predicted_labels == labels).sum().item()
    total_predictions = labels.size(0)
    accuracy = correct_predictions / total_predictions * 100
    d_aft_softmax=max(d_aft_softmax,(threshold,accuracy),key=lambda x:x[1])
print(f"d_bef_softmax: {d_bef_softmax}\nd_aft_softmax: {d_aft_softmax}")

/tmp/ipykernel_31073/257576794.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  median_entropy = torch.median(torch.tensor(entropy_array))
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:05<00:00, 1749.84it/s]


d_bef_softmax: (tensor(0.1440, device='cuda:0'), 85.72999999999999)
d_aft_softmax: (tensor(0.0845, device='cuda:0'), 85.65)


In [11]:
exp_save_file['Dynamic_No_Scaling'] = Dynamic_No_Scaling

##### 동적 앙상블할 때 각 샘플들마다 어떤 출구들로 앙상블하는지 찍기 //시각화는 나중에

In [12]:
threshold = d_bef_softmax[0]
mask = (entropy_array[:, :] <= threshold).to(device)

indices = mask.nonzero()
column_indices = defaultdict(list)
for row, col in indices:
    column_indices[col.item()].append(row.item())

# 각 샘플별로 앙상블할 exit들의 인덱스를 저장
ensemble_exits_per_sample =[[] for _ in range(len(labels_list))]
for col in sorted(column_indices.keys()):
    ensemble_exits_per_sample[col] = column_indices[col]

USE_LAST_EXIT = 1
if USE_LAST_EXIT:
    # 앙상블할때 엔트로피가 임계값 이하인 exit가 하나도 없는 경우 마지막 exit의 추론 결과를 사용하도록 설정
    for i in range(len(labels_list)):
        if len(ensemble_exits_per_sample[i])==0:
            ensemble_exits_per_sample[i].append(exit_num-1) #10
else:
    # 엔트로피가 임계값 이하인 exit가 없는 경우, 가장 작은 엔트로피를 가진 exit 선택
    for i in range(len(labels_list)):
        if len(ensemble_exits_per_sample[i]) == 0:
            # i번째 열(sample)의 모든 엔트로피 값
            column_entropies = entropy_array[:, i]
            # 가장 작은 엔트로피를 가진 exit의 인덱스
            min_entropy_exit = torch.argmin(column_entropies)
            ensemble_exits_per_sample[i].append(min_entropy_exit.item())

exp_save_file[f'choosen_exit_distribution'] = ensemble_exits_per_sample
ensemble_exits_per_sample

[[10],
 [9, 10],
 [10],
 [6, 7, 8, 9, 10],
 [2, 3, 4, 6, 7, 8, 9, 10],
 [10],
 [8, 9, 10],
 [9, 10],
 [4, 7, 9, 10],
 [6, 7, 8, 9, 10],
 [9, 10],
 [9, 10],
 [6, 8, 9, 10],
 [3, 4, 5, 6, 7, 8, 9, 10],
 [3, 4, 5, 6, 7, 8, 9, 10],
 [2, 3, 4, 5, 6, 7, 8, 9, 10],
 [2, 3, 4, 5, 6, 7, 8, 9, 10],
 [3, 4, 9, 10],
 [2, 3, 4, 5, 6, 7, 8, 9, 10],
 [2, 3, 4, 5, 6, 7, 8, 9, 10],
 [3, 5, 6, 7, 8, 9, 10],
 [4, 5, 6, 7, 8, 9, 10],
 [3, 4, 5, 8, 9, 10],
 [10],
 [2, 3, 4, 5, 7, 8, 9, 10],
 [3, 4, 5, 6, 7, 8, 9, 10],
 [2, 3, 4, 5, 6, 7, 8, 9, 10],
 [2, 3, 4, 5, 6, 7, 8, 9, 10],
 [4, 5, 6, 7, 8, 9, 10],
 [9, 10],
 [3, 4, 5, 6, 7, 8, 9, 10],
 [8, 9, 10],
 [7, 8, 9, 10],
 [9, 10],
 [8, 9],
 [10],
 [8, 9, 10],
 [7, 9, 10],
 [7, 8, 9, 10],
 [4, 5, 6, 7, 8, 9, 10],
 [9, 10],
 [8, 9, 10],
 [3, 4, 5, 8, 9, 10],
 [8, 9, 10],
 [2, 3, 4, 5, 6, 7, 8, 9, 10],
 [4, 5, 6, 7, 8, 9, 10],
 [4, 7, 8, 9, 10],
 [2, 5, 6, 7, 8, 9, 10],
 [9, 10],
 [9],
 [5, 6, 7, 8, 9, 10],
 [4, 6, 7, 8, 9, 10],
 [2, 3, 4, 5, 6, 7, 8, 9, 10],
 

##### Temperature Scaling 한 각 출구들에서의 T 값

In [13]:
t_scalers = torch.load(t_scalers_path)
ts_output_tensor = torch.zeros_like(output_tensor)
print(*[f"{round(t_scalers[i].temperature.item(),3)}" for i in range(exit_num)])
with torch.no_grad():
    for i in range(exit_num):
        t_scalers[i].to(device)
        t_scalers[i].eval()
        ts_output_tensor[i] = t_scalers[i](output_tensor[i])

0.829 0.769 0.944 0.95 0.978 0.996 1.008 1.091 1.159 1.952 1.538


/tmp/ipykernel_31073/4070873290.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  t_scalers = torch.load(t_scalers_path)


##### TS동적 앙상블: 모든 exit에서 특정 엔트로피 이하인 것들을 앙상블 하고 소프트맥스 진행하고 맥스값으로 추론

In [14]:
# 모든 exit에서 특정 엔트로피 이하인 것들을 모아서 앙상블을 해본다.
# case 2가지 소프트맥스 전에 합칠지 후에 합칠지;;

step_range = 10000
aft_sftmx = F.softmax(ts_output_tensor,dim=2)
entropy_array= torch.tensor(entropy(aft_sftmx.to('cpu'), base=exit_num, axis=2))

min_entropy = entropy_array.min()
median_entropy = torch.median(torch.tensor(entropy_array))

step_size = (median_entropy - min_entropy) / step_range
d_bef_softmax=(0,0);d_aft_softmax=(0,0)
for mul in tqdm(range(step_range)):
    threshold = min_entropy + mul * step_size
    mask = (entropy_array[:, :] <= threshold).to(device)
    
    column_sums = mask.sum(dim=0)  
    zero_columns = (column_sums == 0)

    last_row = torch.zeros_like(mask)
    last_row[-1, :] = 1
    mask = mask | (last_row & zero_columns)

    mask.unsqueeze_(dim=2)
    masked_array=mask*output_tensor

    # sum before softmax (best acc: 87.75)
    ensemble_logits = masked_array.sum(axis=0)
    ensemble_probabilities = F.softmax(ensemble_logits, dim=1)
    _, predicted_labels = torch.max(ensemble_probabilities, dim=1)
    correct_predictions = (predicted_labels == labels).sum().item()
    total_predictions = labels.size(0)
    accuracy = correct_predictions / total_predictions * 100
    d_bef_softmax=max(d_bef_softmax,(threshold,accuracy),key=lambda x:x[1])

    # sum after softmax (best acc: 87.75)
    softmax_vector_list = F.softmax(masked_array, dim=2)
    ensemble_probabilities = softmax_vector_list.sum(axis=0)
    _, predicted_labels = torch.max(ensemble_probabilities, dim=1)
    correct_predictions = (predicted_labels == labels).sum().item()
    total_predictions = labels.size(0)
    accuracy = correct_predictions / total_predictions * 100
    d_aft_softmax=max(d_aft_softmax,(threshold,accuracy),key=lambda x:x[1])
print(f"d_bef_softmax: {d_bef_softmax}\nd_aft_softmax: {d_aft_softmax}")
Dynamic_TS_Scaling = 0
Dynamic_TS_Scaling = max(Dynamic_TS_Scaling, d_bef_softmax[1], d_aft_softmax[1])

/tmp/ipykernel_31073/3300391826.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  median_entropy = torch.median(torch.tensor(entropy_array))
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:06<00:00, 1447.96it/s]

d_bef_softmax: (tensor(0.0180), 86.25)
d_aft_softmax: (tensor(0.0180), 86.25)


##### TS동적 앙상블: 모든 exit에서 특정 엔트로피 이하인 것들을 앙상블 하고 소프트맥스 진행하고 맥스값으로 추론 임계값 이하가 없으면 제일 엔트로피 낮은거

In [15]:
# 모든 exit에서 특정 엔트로피 이하인 것들을 모아서 앙상블을 해본다.
# case 2가지 소프트맥스 전에 합칠지 후에 합칠지;;

step_range = 10000
aft_sftmx = F.softmax(ts_output_tensor,dim=2)
entropy_array= torch.tensor(entropy(aft_sftmx.to('cpu'), base=exit_num, axis=2)).to(device)

min_entropy = entropy_array.min()
median_entropy = torch.median(torch.tensor(entropy_array))

step_size = (median_entropy - min_entropy) / step_range
d_bef_softmax=(0,0);d_aft_softmax=(0,0)

########################################
# 각 열의 최소값 인덱스 찾기
min_indices = entropy_array.argmin(dim=0)
# 2D 행렬 생성 및 초기화 (모든 값 0)
result_matrix = torch.zeros_like(entropy_array.int())
# 최소값 인덱스 위치에 1 설정
for col, row in enumerate(min_indices):
    result_matrix[row, col] = 1
########################################

for mul in tqdm(range(step_range)):
    threshold = min_entropy + mul * step_size
    mask = (entropy_array[:, :] <= threshold).to(device)
    
    column_sums = mask.sum(dim=0)
    zero_columns = (column_sums == 0)
    mask = mask | (result_matrix & zero_columns)

    mask.unsqueeze_(dim=2)
    masked_array=mask*output_tensor

    # sum before softmax (best acc: 87.75)
    ensemble_logits = masked_array.sum(axis=0)
    ensemble_probabilities = F.softmax(ensemble_logits, dim=1)
    _, predicted_labels = torch.max(ensemble_probabilities, dim=1)
    correct_predictions = (predicted_labels == labels).sum().item()
    total_predictions = labels.size(0)
    accuracy = correct_predictions / total_predictions * 100
    d_bef_softmax=max(d_bef_softmax,(threshold,accuracy),key=lambda x:x[1])

    # sum after softmax (best acc: 87.75)
    softmax_vector_list = F.softmax(masked_array, dim=2)
    ensemble_probabilities = softmax_vector_list.sum(axis=0)
    _, predicted_labels = torch.max(ensemble_probabilities, dim=1)
    correct_predictions = (predicted_labels == labels).sum().item()
    total_predictions = labels.size(0)
    accuracy = correct_predictions / total_predictions * 100
    d_aft_softmax=max(d_aft_softmax,(threshold,accuracy),key=lambda x:x[1])
print(f"d_bef_softmax: {d_bef_softmax}\nd_aft_softmax: {d_aft_softmax}")
Dynamic_TS_Scaling = max(Dynamic_TS_Scaling, d_bef_softmax[1], d_aft_softmax[1])

/tmp/ipykernel_31073/2342929353.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  median_entropy = torch.median(torch.tensor(entropy_array))
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:05<00:00, 1750.46it/s]

d_bef_softmax: (tensor(0.3278, device='cuda:0'), 85.21)
d_aft_softmax: (tensor(0.2540, device='cuda:0'), 85.04)


In [16]:
exp_save_file["Dynamic_TS_Scaling"] = Dynamic_TS_Scaling

##### 각 출구마다 라벨 별로 Precision 확인 후 각 출구의 mAP 확인

In [17]:
exits_precision = np.array([[0.0] * dataset_outdim[data_choice] for _ in range(exit_num)])
for exit in [i for i in range(exit_num)]:
    label_correct = defaultdict(int)
    label_total = defaultdict(int)
    
    new_output_list = output_tensor[exit]
    logits = new_output_list
    probabilities = F.softmax(logits, dim=1)
    _, predicted = torch.max(probabilities, dim=1)
    for label, pred in zip(labels, predicted):
            if label == pred:
                label_correct[label.item()] += 1
            label_total[label.item()] += 1
    
    label_accuracy = {label: round(label_correct[label] / label_total[label] * 100, 2) for label in range(dataset_outdim[data_choice])}
    exits_precision[exit] = [label_accuracy[label] for label in range(dataset_outdim[data_choice])]
exp_save_file[f'exits_precision'] = exits_precision
exits_precision

array([[75., 54., 39., ..., 36., 24., 32.],
       [77., 63., 43., ..., 47., 26., 41.],
       [81., 80., 50., ..., 51., 32., 67.],
       ...,
       [95., 87., 73., ..., 82., 68., 84.],
       [96., 90., 75., ..., 84., 73., 88.],
       [95., 94., 77., ..., 92., 76., 90.]])

In [18]:
m_scalers = torch.load(m_scalers_path)
ms_output_tensor = torch.zeros_like(output_tensor)
with torch.no_grad():
    for i in range(exit_num):
        m_scalers[i].to(device)
        m_scalers[i].eval()
        ms_output_tensor[i] = m_scalers[i](output_tensor[i])

/tmp/ipykernel_31073/2047556124.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  m_scalers = torch.load(m_scalers_path)


In [19]:
# 모든 exit에서 특정 엔트로피 이하인 것들을 모아서 앙상블을 해본다.
# case 2가지 소프트맥스 전에 합칠지 후에 합칠지;;

step_range = 10000
aft_sftmx = F.softmax(ms_output_tensor,dim=2)
entropy_array= torch.tensor(entropy(aft_sftmx.to('cpu'), base=exit_num, axis=2))

min_entropy = entropy_array.min()
median_entropy = torch.median(torch.tensor(entropy_array))

step_size = (median_entropy - min_entropy) / step_range
d_bef_softmax=(0,0);d_aft_softmax=(0,0)
for mul in tqdm(range(step_range)):
    threshold = min_entropy + mul * step_size
    mask = (entropy_array[:, :] <= threshold).to(device)
    
    column_sums = mask.sum(dim=0)  
    zero_columns = (column_sums == 0)

    last_row = torch.zeros_like(mask)
    last_row[-1, :] = 1
    mask = mask | (last_row & zero_columns)

    mask.unsqueeze_(dim=2)
    masked_array=mask*output_tensor

    # sum before softmax (best acc: 87.75)
    ensemble_logits = masked_array.sum(axis=0)
    ensemble_probabilities = F.softmax(ensemble_logits, dim=1)
    _, predicted_labels = torch.max(ensemble_probabilities, dim=1)
    correct_predictions = (predicted_labels == labels).sum().item()
    total_predictions = labels.size(0)
    accuracy = correct_predictions / total_predictions * 100
    d_bef_softmax=max(d_bef_softmax,(threshold,accuracy),key=lambda x:x[1])

    # sum after softmax (best acc: 87.75)
    softmax_vector_list = F.softmax(masked_array, dim=2)
    ensemble_probabilities = softmax_vector_list.sum(axis=0)
    _, predicted_labels = torch.max(ensemble_probabilities, dim=1)
    correct_predictions = (predicted_labels == labels).sum().item()
    total_predictions = labels.size(0)
    accuracy = correct_predictions / total_predictions * 100
    d_aft_softmax=max(d_aft_softmax,(threshold,accuracy),key=lambda x:x[1])
Dynamic_MS_Scaling = 0
Dynamic_MS_Scaling = max(Dynamic_MS_Scaling, d_bef_softmax[1], d_aft_softmax[1])

/tmp/ipykernel_31073/2792468018.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  median_entropy = torch.median(torch.tensor(entropy_array))
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:06<00:00, 1441.60it/s]


In [20]:
exp_save_file["Dynamic_MS_Scaling"] = Dynamic_MS_Scaling

In [21]:
torch.save(exp_save_file, exp_result_path)